In [1]:
#Preprocessing Step 1 (txt to CSV)
"""
Cartesian
Name,X,Y,Z,AGL,INFO,SUBFOLDER,DOME,CONE
Site1_2_P_1_S,452621.3740,7183555.1190,145.510,0.000,,(null),N,N
Site1_2_P_1_F,452621.8271,7183554.7090,145.490,0.000,,(null),N,N
Site1_2_P_2_S,452620.6910,7183557.2040,145.490,0.000,,(null),N,N
Site1_2_P_2_F,452621.4940,7183556.6130,145.530,-0.000,,(null),N,N
Site1_2_P_9_S,452601.8660,7183612.7730,144.930,-0.000,,(null),N,N

to

Damage_ID,X,Y,Z,Label
Site1_2_P_1_S,452621.374,7183555.119,145.51,P
Site1_2_P_1_F,452621.8271,7183554.709,145.49,P
Site1_2_P_2_S,452620.691,7183557.204,145.49,P
Site1_2_P_2_F,452621.494,7183556.613,145.53,P
Site1_2_P_3_S,452607.1435,7183603.6467,145.1,P
Site1_2_P_3_F,452606.7625,7183603.7849,145.12,P
"""
import os
import csv
import re

input_folder = r"C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables"
output_folder = os.path.join(input_folder, "cleancsvs")

os.makedirs(output_folder, exist_ok=True)

LABELS = ["P", "C", "E", "FL"]


def find_label(name):
    """Find P, C, E, Fl ANYWHERE in the name."""
    n = name.upper()

    # Priority: Fl first (two letters)
    if "FL" in n:
        return "Fl"

    # Single-letter labels
    for L in ["P", "C", "E"]:
        if L in n:
            return L

    return None


def has_final_SF(name):
    """
    Check if the name ends with S or F ANYWHERE before numbers.
    We look for last alphabetic character.
    """
    letters = re.findall("[A-Za-z]", name)
    if not letters:
        return False

    last = letters[-1].upper()
    return last in ["S", "F"]


def clean_name(name):
    """Normalize double underscores etc."""
    name = name.replace("__", "_")
    name = re.sub(r"_+", "_", name)
    return name.strip()


for filename in os.listdir(input_folder):
    if not filename.lower().endswith(".txt"):
        continue

    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, filename.replace(".txt", ".csv"))

    saved = 0
    rejected = 0
    rows = []

    with open(input_path, "r", errors="ignore") as f:
        for line in f:
            parts = [p.strip() for p in line.split(",")]

            if len(parts) < 4:
                rejected += 1
                continue

            name = clean_name(parts[0])

            # Extract coordinates
            try:
                X = float(parts[1])
                Y = float(parts[2])
                Z = float(parts[3])
            except:
                rejected += 1
                continue

            # Find label anywhere in name
            label = find_label(name)
            if label is None:
                rejected += 1
                continue

            # Check F / S anywhere at end
            if not has_final_SF(name):
                rejected += 1
                continue

            rows.append([name, X, Y, Z, label])
            saved += 1

    if rows:
        with open(output_path, "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Damage_ID", "X", "Y", "Z", "Label"])
            writer.writerows(rows)
        print(f"[OK] {filename} -> {output_path}  (total={saved+rejected}, saved={saved}, rejected={rejected})")
    else:
        print(f"[WARNING] All lines rejected in {filename} (total={saved+rejected})")

[OK] Site1_0.txt -> C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\Site1_0.csv  (total=4, saved=2, rejected=2)
[OK] Site1_1.txt -> C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\Site1_1.csv  (total=42, saved=40, rejected=2)
[OK] Site1_10.txt -> C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\Site1_10.csv  (total=88, saved=76, rejected=12)
[OK] Site1_11.txt -> C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\Site1_11.csv  (total=64, saved=62, rejected=2)
[OK] Site1_12.txt -> C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\Site1_12.csv  (total=72, saved=70, rejected=2)
[OK] Site1_13.txt -> C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\Site1_13.csv  (total=20, saved=18, rejected=

In [2]:
#preprocessing step 2 CSV 2 lines to 1 line lable
"""
Damage_ID,X,Y,Z,Label
Site1_2_P_1_S,452621.374,7183555.119,145.51,P
Site1_2_P_1_F,452621.8271,7183554.709,145.49,P
Site1_2_P_2_S,452620.691,7183557.204,145.49,P
Site1_2_P_2_F,452621.494,7183556.613,145.53,P
Site1_2_P_3_S,452607.1435,7183603.6467,145.1,P
Site1_2_P_3_F,452606.7625,7183603.7849,145.12,P

to 

Damage_ID,Damage_Type,Xmin,Xmax,Ymin,Ymax,Zmin,Zmax
Site1_2_P_1,P,452621.374,452621.8271,7183554.709,7183555.119,145.49,145.51
Site1_2_P_10,P,452603.178,452604.0189,7183613.7414,7183614.308,144.97,144.97
Site1_2_P_11,P,452600.723,452601.5552,7183614.9232,7183615.352,144.94,144.99
Site1_2_P_2,P,452620.691,452621.494,7183556.613,7183557.204,145.49,145.53
Site1_2_P_3,P,452606.7625,452607.1435,7183603.6467,7183603.7849,145.1,145.12
Site1_2_P_4,P,452604.2179,452604.7857,7183605.7,7183606.4693,144.98,144.99
"""
import pandas as pd
import os
import re

# ---------------------------------------------------
# CONFIG
# ---------------------------------------------------
input_folder = r"C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs"          
output_folder = r"C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\improved_csvs"      
combined_output = os.path.join(output_folder, "combined_output.csv")  # one combined file
summary_output = os.path.join(output_folder, "summary_per_site.csv")  # summary table

os.makedirs(output_folder, exist_ok=True)

# ---------------------------------------------------
# HELPER FUNCTIONS
# ---------------------------------------------------
def get_base_id(dmg_id):
    """Remove trailing _S or _F to get base Damage_ID"""
    return re.sub(r'_(S|F)$', '', dmg_id)

def get_damage_type(dmg_id):
    """Detect Fl first, then P, C, E anywhere in ID"""
    for t in ["Fl", "P", "C", "E"]:
        if t in dmg_id:
            return t
    return "Unknown"

# ---------------------------------------------------
# PROCESS FILES
# ---------------------------------------------------
all_rows = []
summary_list = []

csv_files = [f for f in os.listdir(input_folder) if f.lower().endswith(".csv")]

for fname in csv_files:
    in_path = os.path.join(input_folder, fname)
    out_path = os.path.join(output_folder, fname)

    df = pd.read_csv(in_path)
    df.columns = ["Damage_ID", "X", "Y", "Z", "Label"]

    df["base_id"] = df["Damage_ID"].apply(get_base_id)
    df["damage_type"] = df["Damage_ID"].apply(get_damage_type)

    rows = []

    # Count per damage type for this file
    type_counts = {"P": 0, "C": 0, "E": 0, "Fl": 0}

    for base_id, g in df.groupby("base_id"):
        dmg_type = g["damage_type"].iloc[0]

        xmin, xmax = g["X"].min(), g["X"].max()
        ymin, ymax = g["Y"].min(), g["Y"].max()
        zmin, zmax = g["Z"].min(), g["Z"].max()

        row = [base_id, dmg_type, xmin, xmax, ymin, ymax, zmin, zmax]
        rows.append(row)
        all_rows.append([fname] + row)

        # Update counts
        if dmg_type in type_counts:
            type_counts[dmg_type] += 1

    total = sum(type_counts.values())
    summary_list.append({
        "Site": fname.replace(".csv", ""),
        "P_BBoxes": type_counts["P"],
        "C_BBoxes": type_counts["C"],
        "E_BBoxes": type_counts["E"],
        "Fl_BBoxes": type_counts["Fl"],
        "Total": total
    })

    # Save per-file bounding boxes
    out_df = pd.DataFrame(
        rows,
        columns=["Damage_ID", "Damage_Type", "Xmin", "Xmax", "Ymin", "Ymax", "Zmin", "Zmax"]
    )
    out_df.to_csv(out_path, index=False)

# ---------------------------------------------------
# COMBINED CSV
# ---------------------------------------------------
combined_df = pd.DataFrame(
    all_rows,
    columns=["Source_File", "Damage_ID", "Damage_Type",
             "Xmin", "Xmax", "Ymin", "Ymax", "Zmin", "Zmax"]
)
combined_df.to_csv(combined_output, index=False)

# ---------------------------------------------------
# SUMMARY TABLE
# ---------------------------------------------------
summary_df = pd.DataFrame(summary_list)

# Add total row
totals = summary_df[["P_BBoxes","C_BBoxes","E_BBoxes","Fl_BBoxes","Total"]].sum()
totals_row = pd.DataFrame([["Total", totals["P_BBoxes"], totals["C_BBoxes"], totals["E_BBoxes"], totals["Fl_BBoxes"], totals["Total"]]],
                          columns=summary_df.columns)
summary_df = pd.concat([summary_df, totals_row], ignore_index=True)

summary_df.to_csv(summary_output, index=False)

print(f"✔ Done. Individual CSVs saved in: {output_folder}")
print(f"✔ Combined CSV saved as: {combined_output}")
print(f"✔ Summary CSV saved as: {summary_output}")

✔ Done. Individual CSVs saved in: C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\improved_csvs
✔ Combined CSV saved as: C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\improved_csvs\combined_output.csv
✔ Summary CSV saved as: C:\Users\umair.muhammad\Documents\PhD\Research Work\FedLearn\training\All_Nome\txt_lables\cleancsvs\improved_csvs\summary_per_site.csv


In [8]:
"""
Pre-Processing Step 3
Murging bounding bos and Laz to get Pickle files

Found 6 bounding boxes
pc_id=48: 843050 points
  class 0 (unlabeled): 842888 points
  class 1 (P): 162 points

  ==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====
Class     Label     Points         
-----------------------------------
1         P         3973492        
2         C         144231         
3         E         10856          
4         Fl        58100          
0         unlabeled 230745632      

Total points across all processed LAS files: 234932311
"""

import os
import laspy
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from sklearn.cluster import DBSCAN

############################################
# USER CONFIG
############################################

LAS_FOLDER = "C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/LAS_Files"
CSV_FOLDER = "C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs"
OUTPUT_FOLDER = "C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/Full_Pickle_dataset"

# Label mapping
CLASS_MAP = {
    "P": 1,       # pothole
    "C": 2,       # crack
    "E": 3,       # edge crack
    "Fl": 4,      # flushed out
}

# Bounding box margin (meters)
BOX_MARGIN = 0.3

# DBSCAN expansion (meters)
USE_DBSCAN = True
DBSCAN_EPS = 0.2
DBSCAN_MIN_SAMPLES = 1

############################################
# UTILS
############################################

def load_damage_csv(csv_path):
    """Load bounding boxes from CSV"""
    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]  # remove spaces
    boxes = []
    for _, row in df.iterrows():
        boxes.append({
            "bbox": (row["Xmin"], row["Xmax"], row["Ymin"], row["Ymax"], row["Zmin"], row["Zmax"]),
            "type": row["Damage_Type"].strip()
        })
    return boxes

def label_points(points, boxes, use_dbscan=True):
    """Assign class labels to points given bounding boxes"""
    labels = np.zeros(points.shape[0], dtype=np.int32)
    for box in boxes:
        xmin, xmax, ymin, ymax, zmin, zmax = box["bbox"]
        cls = CLASS_MAP.get(box["type"], 0)

        # Inflate bounding box
        mask = (
            (points[:, 0] >= xmin - BOX_MARGIN) & (points[:, 0] <= xmax + BOX_MARGIN) &
            (points[:, 1] >= ymin - BOX_MARGIN) & (points[:, 1] <= ymax + BOX_MARGIN) &
            (points[:, 2] >= zmin - BOX_MARGIN) & (points[:, 2] <= zmax + BOX_MARGIN)
        )
        labels[mask] = cls

    # Optional DBSCAN expansion
    if use_dbscan:
        for cls_val in np.unique(labels):
            if cls_val == 0:
                continue
            idx = np.where(labels == cls_val)[0]
            if len(idx) == 0:
                continue
            coords = points[idx, :3]
            db = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MIN_SAMPLES).fit(coords)
            if hasattr(db, 'core_sample_indices_') and len(db.core_sample_indices_) > 0:
                expanded_idx = idx[db.core_sample_indices_]
                labels[expanded_idx] = cls_val

    return labels

############################################
# MAIN LOOP
############################################

os.makedirs(OUTPUT_FOLDER, exist_ok=True)
pc_id = 0

# Summary counters
total_class_counts = {cls: 0 for cls in CLASS_MAP.values()}
total_class_counts[0] = 0  # for unlabeled points

for las_file in tqdm(os.listdir(LAS_FOLDER)):
    if not las_file.lower().endswith(".las"):
        continue

    las_path = os.path.join(LAS_FOLDER, las_file)
    
    # Match CSV for this LAS file
    site_name = os.path.splitext(las_file)[0]  # e.g., Site1_7
    csv_path = os.path.join(CSV_FOLDER, f"{site_name}.csv")

    if not os.path.exists(csv_path):
        print(f"\nSkipping {las_file}: No CSV found -> {csv_path}")
        continue  # skip this LAS file entirely

    print(f"\nProcessing {las_file} -> {csv_path}")

    # read LAS
    las = laspy.read(las_path)
    x = las.x
    y = las.y
    z = las.z

    # Use intensity as feature (or RGB if available)
    intensity = las.intensity.astype(np.float32)
    rgb = np.stack([intensity, intensity, intensity], axis=1)

    pts = np.vstack([x, y, z]).T
    pts_full = np.hstack([pts, rgb])  # (N, 6)

    # load bounding boxes
    boxes = load_damage_csv(csv_path)
    print(f"Found {len(boxes)} bounding boxes")

    # assign labels with DBSCAN expansion
    labels = label_points(pts, boxes, use_dbscan=USE_DBSCAN)
    labels = labels.reshape(-1, 1)

    # Update summary counts
    unique, counts = np.unique(labels, return_counts=True)
    for u, c in zip(unique, counts):
        total_class_counts[u] += c

    # final (N,7) format
    final_pc = np.hstack([pts_full, labels])

    # save in required directory structure
    pc_id += 1
    out_dir = os.path.join(OUTPUT_FOLDER, f"pc_id={pc_id}")
    os.makedirs(out_dir, exist_ok=True)
    os.makedirs(os.path.join(out_dir, "metadata"), exist_ok=True)

    # save pc
    with open(os.path.join(out_dir, "pc.pickle"), "wb") as f:
        pickle.dump(final_pc.astype(np.float32), f)

    # save metadata
    metadata = {
        "num_points": final_pc.shape[0],
        "file_source": las_file,
        "classes": CLASS_MAP
    }
    with open(os.path.join(out_dir, "metadata/metadata.pickle"), "wb") as f:
        pickle.dump(metadata, f)

    # print per-file summary
    print(f"pc_id={pc_id}: {len(labels)} points")
    for u, c in zip(unique, counts):
        cls_name = "unlabeled" if u == 0 else [k for k,v in CLASS_MAP.items() if v==u][0]
        print(f"  class {u} ({cls_name}): {c} points")

# ------------------------------
# Final summary
# ------------------------------
print("\n✔ Conversion complete!")
print("\n==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====")
print(f"{'Class':<10}{'Label':<10}{'Points':<15}")
print("-" * 35)
for cls, count in total_class_counts.items():
    cls_name = "unlabeled" if cls == 0 else [k for k,v in CLASS_MAP.items() if v==cls][0]
    print(f"{cls:<10}{cls_name:<10}{count:<15}")

total_points = sum(total_class_counts.values())
print(f"\nTotal points across all processed LAS files: {total_points}")


  0%|          | 0/149 [00:00<?, ?it/s]


Processing Site1_0.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_0.csv
Found 1 bounding boxes


  1%|          | 1/149 [00:00<00:39,  3.78it/s]

pc_id=1: 1917694 points
  class 0 (unlabeled): 1916652 points
  class 1 (P): 1042 points

Processing Site1_1.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_1.csv
Found 20 bounding boxes
pc_id=2: 306546 points
  class 0 (unlabeled): 298388 points
  class 1 (P): 8158 points

Processing Site1_10.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\Site1_10.csv
Found 38 bounding boxes


  2%|▏         | 3/149 [00:00<00:29,  5.01it/s]

...

pc_id=144: 2732103 points
  class 0 (unlabeled): 2692812 points
  class 1 (P): 39291 points

Processing SiteV2_8.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\SiteV2_8.csv
Found 30 bounding boxes


 99%|█████████▉| 148/149 [01:40<00:02,  2.26s/it]

pc_id=145: 4004789 points
  class 0 (unlabeled): 3954853 points
  class 1 (P): 48655 points
  class 4 (Fl): 1281 points

Processing SiteV2_9.las -> C:/Users/umair.muhammad/Documents/PhD/Research Work/FedLearn/training/All_Nome/txt_lables/cleancsvs/improved_csvs\SiteV2_9.csv
Found 34 bounding boxes


100%|██████████| 149/149 [01:41<00:00,  1.47it/s]

pc_id=146: 4972295 points
  class 0 (unlabeled): 4929349 points
  class 1 (P): 37879 points
  class 4 (Fl): 5067 points

✔ Conversion complete!

==== TOTAL POINTS PER CLASS ACROSS ALL FILES ====
Class     Label     Points         
-----------------------------------
1         P         3973492        
2         C         144231         
3         E         10856          
4         Fl        58100          
0         unlabeled 230745632      

Total points across all processed LAS files: 234932311


In [9]:
import pandas as pd

# -------------------------------------------
# Your damage counts (replace if needed)
# -------------------------------------------
data = {
    "Category": ["P", "C", "E", "Fl", "Unlabeled"],
    "Points": [3973492, 144231, 10856, 58100, 230745632]
}

total_points = 234932311  # total points across all LAS files

# -------------------------------------------
# Create dataframe
# -------------------------------------------
df = pd.DataFrame(data)

# Calculate percentage damage
df["Percent"] = (df["Points"] / total_points) * 100

# -------------------------------------------
# Display clean formatted table
# -------------------------------------------
print("\nDamage Breakdown (Percent of Total Points)\n")
print(df.to_string(index=False, formatters={
    "Points": "{:,}".format,
    "Percent": "{:.4f}%".format
}))



Damage Breakdown (Percent of Total Points)

 Category      Points  Percent
        P   3,973,492  1.6913%
        C     144,231  0.0614%
        E      10,856  0.0046%
       Fl      58,100  0.0247%
Unlabeled 230,745,632 98.2179%
